# Tarea 2:

Instrucciones de ejecución:
1. Cambiar la dirección de los archivos en el caso del coloreado. Como se corrió de manera local se tiene la dirección de los archivos en mi computadora.
2. Para el caso de Minimax hay que clonar el repositorio y mudarse a ese directorio. (Cosa que se hace en el notebook, así que evitar clonar dos veces el repositorio es importante)

## Backtracking

La implementación de este algoritmo se ajustará al problema en cuestión, por lo que primero tendremos que representar los problemas de N-Reinas al igual que los de coloreado antes de desarrollar los algoritmos mismos.

### N_Reinas

In [18]:
# Las funciones necesarias para la implementación sobre el problema de la N-Reinas

def valid(tablero, fila, columnas, n):
    """
    Verifica si el estado en observación es un estado aceptable para agregar una reina.
    Es equivalente a una implementación equivalente a Backtracking + Revisión hacia adelante , ya que
    antes de colocar la reina verifica que la posición donde busca colocarla sea la óptima. 

    Está altamente ajustada a la representación del tablero usada. Difícil de extender a una CSP general.
    """
    
    # Verificación por filas    
    for i in range(columnas):
        if tablero[fila][i] == 'R':
            #print("Fila")
            return False

    # Verificación por diagonal superior
    for i, j in zip(range(fila, -1, -1), range(columnas, -1, -1)):
        if tablero[i][j] == 'R':
            #print("Diag Sup")
            return False

    # Verificación por diagonal inferior
    for i, j in zip(range(fila, n, 1), range(columnas, -1, -1)):
        if tablero[i][j] == 'R':
            #print("Diag Inf")
            return False
            
    return True


#Función principal
def n_reinas(n):
    """
    El tablero se representará como un arreglo de n entradas donde cada entrada es un arreglo de n entradas. 
    Para denotar una casilla vacía se usará "-", mientras que para denotar una reina se usará "R".
    Esta función agrega reinas una por una cuidando las restricciones. Itera sobre columnas.
    Verificar que no sea problemática la nueva reina se hace en la función previa valid(tablero, fila, columnas, n)
    """
    
    tablero = [['-' for i in range(n)] for i in range(n)]
    result = []

    def backtrack(tablero, columnas):
        if columnas == n:
            result.append([', '.join(fila) for fila in tablero])
            return

        for i in range(n):
            if valid(tablero, i, columnas, n):
                tablero[i][columnas] = 'R'
                backtrack(tablero, columnas + 1)
                tablero[i][columnas] = '-'

    backtrack(tablero, 0)
    return result

# Despliegue de resultados
def full_n_reinas(n):
    """
    Solo ejecuta las funciones previas con una n definida por el usuario.
    """
    aux = n_reinas(n)
    print(f"Cantidad de reinas: {n}.")
    print(f"Cantidad de soluciones encontradas: {len(aux)}.")
    print("Primera solución encontrada:")
    for i in aux[0]:
        print(i)

In [25]:
%%time
# Ejemplos
full_n_reinas(12)
full_n_reinas(8)

# Para n=100 no logré que acabara. Corra esta variable bajo su propio riesgo.
#full_n_reinas(100)

Cantidad de reinas: 12.
Cantidad de soluciones encontradas: 14200.
Primera solución encontrada:
R, -, -, -, -, -, -, -, -, -, -, -
-, -, -, -, -, -, -, -, R, -, -, -
-, R, -, -, -, -, -, -, -, -, -, -
-, -, -, -, -, -, -, -, -, -, -, R
-, -, R, -, -, -, -, -, -, -, -, -
-, -, -, -, -, -, R, -, -, -, -, -
-, -, -, -, -, -, -, -, -, R, -, -
-, -, -, R, -, -, -, -, -, -, -, -
-, -, -, -, -, -, -, -, -, -, R, -
-, -, -, -, R, -, -, -, -, -, -, -
-, -, -, -, -, -, -, R, -, -, -, -
-, -, -, -, -, R, -, -, -, -, -, -
Cantidad de reinas: 8.
Cantidad de soluciones encontradas: 92.
Primera solución encontrada:
R, -, -, -, -, -, -, -
-, -, -, -, -, -, R, -
-, -, -, -, R, -, -, -
-, -, -, -, -, -, -, R
-, R, -, -, -, -, -, -
-, -, -, R, -, -, -, -
-, -, -, -, -, R, -, -
-, -, R, -, -, -, -, -
CPU times: total: 6.41 s
Wall time: 6.42 s


### Coloreado

In [32]:
import networkx as nx #Para facilitar la representación usando grafos.
import time

# Función de procesamiento de texto 
def leer_txt(name_txt):
    """
    Abre y lee el archivo en cuestión.
    Se pasa a una gráfica de nx.Graph() para evitar tener que hacer una implementación particular dependiente del problema.
    (Esta representación es ideal en caso de que se busque implementar para CSP generalizados).
    """
    with open(name_txt, 'r') as file:
        lines = file.readlines()

    n_vertex, n_edge = map(int, lines[0].strip().split())
    grafo = nx.Graph()

    for linea in lines[1:]:
        a, b = map(int, linea.strip().split())
        grafo.add_edge(a, b)

    return grafo

#Función de coloreado
def backtracking_coloreado(grafo):
    """
    Dado que estamos trabajando con una gráfica podemos revisar sus vecinos de una manera más accesible.
    Esta función revisa vecinos, verifica condiciones, y colorea. Es backtracking pero aplicada a este
    problema y a estas variables.
    Dentro de esta función se tiene la función color_posible() que verifica la validez del color sobre vértice.
    """
    coloreado = {}
    order_vertex = sorted(grafo.nodes(), key=lambda x: -len(list(grafo.neighbors(x))))

    def color_posible(vertex, color):
        """
        Verifica que un vértice se pueda pintar de cierto color.
        """
        for next in grafo.neighbors(vertex):
            if coloreado.get(next) == color:
                return False
        return True

    for vertex in order_vertex:
        for color in range(len(order_vertex)):
            if color_posible(vertex, color):
                coloreado[vertex] = color
                break

    return coloreado

#Función mostrar grafo coloreado
def color_graph(name_txt):
    """
    Implementación puntual del algoritmo.
    Se presenta una solución encontrada.
    """
    grafo = leer_txt(name_txt)
    resultado = backtracking_coloreado(grafo)

    if resultado:
        num_colores = max(resultado.values()) + 1
        print("Se encontró la siguiente solución:")
        print(f"Número de colores: {num_colores}")
        for vertex, color in resultado.items():
            print(f"Color: {color}, vértice {vertex}")
    else:
        print("Solución no encontrada")

In [34]:
%%time

#Implementación con los datasets necesarios.
#Corre en un tiempo bastante razonable así que no hay problema en ejecutar toda la celda.

nodos50 = r"C:\Users\FLopezP\Desktop\PCIC\IA\Prácticas\gc_50_7 (1)"
color_graph(nodos50)

print("-------------------------------------")
nodos1000 = r"C:\Users\FLopezP\Desktop\PCIC\IA\Prácticas\gc_1000_9 (1)"
color_graph(nodos1000)

Se encontró la siguiente solución:
Número de colores: 18
Color: 0, vértice 11
Color: 1, vértice 31
Color: 2, vértice 38
Color: 3, vértice 40
Color: 4, vértice 36
Color: 4, vértice 3
Color: 1, vértice 10
Color: 3, vértice 33
Color: 5, vértice 35
Color: 6, vértice 18
Color: 2, vértice 48
Color: 6, vértice 0
Color: 7, vértice 4
Color: 7, vértice 22
Color: 8, vértice 23
Color: 9, vértice 32
Color: 9, vértice 43
Color: 8, vértice 19
Color: 8, vértice 21
Color: 10, vértice 27
Color: 1, vértice 34
Color: 10, vértice 37
Color: 5, vértice 39
Color: 11, vértice 47
Color: 12, vértice 28
Color: 0, vértice 5
Color: 11, vértice 6
Color: 13, vértice 9
Color: 5, vértice 12
Color: 11, vértice 13
Color: 14, vértice 14
Color: 15, vértice 17
Color: 2, vértice 29
Color: 3, vértice 30
Color: 12, vértice 49
Color: 11, vértice 1
Color: 10, vértice 24
Color: 13, vértice 25
Color: 14, vértice 41
Color: 7, vértice 2
Color: 16, vértice 8
Color: 15, vértice 15
Color: 0, vértice 26
Color: 0, vértice 7
Color: 13, vé

## Recocido Simulado

Las implementaciones aquí son un poco más similares entre sí, no obstante el problema a resolver define fuertemente el funcionamiento puntual de cada implementación. Sigo sin poder encontrar una forma general de representar los CSP.

### N Reinas

In [59]:
import random
import math
import time

def ataque_reinas(tab):
    """
    Una forma alterna de verificar la validez de los lugares a analizar.
    """
    n = len(tab)
    n_ataques = 0
    for i in range(n):
        for j in range(i + 1, n):
            if tab[i] == tab[j] or abs(tab[i] - tab[j]) == j - i:
                n_ataques += 1
    return n_ataques

def REC_SIM(reinas, Ti, FE):
    """
    Implementación del Recocido Simulado.
    Los parámetros definidos por Ti, FE están definidos por parte del usuario.
    """
    new_tab = [random.randint(0, reinas - 1) for _ in range(reinas)]
    new_n_ataques = ataque_reinas(new_tab)
    final_tab = list(new_tab)
    final_n_ataques = new_n_ataques

    T = Ti
    while T > 0:
        """
        Iteración bajo restricción probabilística.
        """
        m = random.randint(0, reinas - 1)
        n = random.randint(0, reinas - 1)
        next_tab = list(new_tab)
        next_tab[m] = n
        next_n_ataques = ataque_reinas(next_tab)

        delta_n_ataques = next_n_ataques - new_n_ataques
        if delta_n_ataques < 0 or random.random() < math.exp(-delta_n_ataques / T):
            new_tab = list(next_tab)
            new_n_ataques = next_n_ataques

        if new_n_ataques < final_n_ataques:
            final_tab = list(new_tab)
            final_n_ataques = new_n_ataques

        T *= FE # Reajuste

    return final_tab, final_n_ataques

def recocido_n_reinas(n, ti, fe):
    """
    Implementación completa e impresión de resultados.
    """
    sol, ataques = REC_SIM(n, ti, fe)
    if ataques == 0:
        print("Se encontró la siguiente solución:")
        for i in range(n):
            fila = ['-'] * n
            fila[sol[i]] = 'R'
            print(', '.join(fila))
    else:
        print("No se encontró solución")

CPU times: total: 0 ns
Wall time: 0 ns


In [85]:
%%time

# Dependeindo del valor de probabilidad dado puede tomar muuuuuuuuuuuuucho tiempo.
# Con 0.5 corre muy rápido pero no siempre llega a un resultado.
# Pero al cambiar el parámetro de 0.5 a un valor distinto toma demasiado, no estoy seguro qué causa eso.

recocido_n_reinas(8, 1000, 0.5)
#recocido_n_reinas(100, 1000, 0.8) # Correr bajo su propio riesgo.

Se encontró la siguiente solución:
-, -, R, -, -, -, -, -
-, -, -, -, -, R, -, -
-, -, -, -, -, -, -, R
R, -, -, -, -, -, -, -
-, -, -, R, -, -, -, -
-, -, -, -, -, -, R, -
-, -, -, -, R, -, -, -
-, R, -, -, -, -, -, -
CPU times: total: 15.6 ms
Wall time: 9 ms


### Coloreado de Grafos

In [87]:
import networkx as nx
import random
import math
import time

# Función leer grafo de archivo txt
# Son diferentes para el caso de backtracking. No obstante son casi iguales
def txt_grafos_recocido(name_txt):
    with open(name_txt, 'r') as archivo:
        lines = archivo.readlines()
        return lines

def crear_graph(lines):
    """
    Nuevamente vamos a crear una gráfica para facilitar el recorrido
    """
    graph = nx.Graph()
    n_vertex = 0
    for i, line in enumerate(lines):
        if i == 0:
            n_vertex, n_edge = map(int, line.split())
        else:
            a, b = map(int, line.split())
            graph.add_edge(a, b)
    return graph, n_vertex

def fun_val_coloreado(graph, coloreado):
    """
    Se cuenta la cantidad de vértices que comparten color.
    """
    val = 0
    for a, b in graph.edges():
        if coloreado[a] == coloreado[b]:
            val += 1
    return val // 2

def new_T(Ti, cont, total_cont):
    """
    Reajuste del recocido en función del estado de la gráfica.
    """
    return Ti * (1 - cont / total_cont)

def random_next(graph, coloreado, n_vertex):
    """
    Elección aleatoria de un nuevo nodo y color.
    """
    nodo = random.choice(list(graph.nodes()))
    color = random.randint(1, n_vertex)
    coloreado[nodo] = color

def REC_SIM(graph, Ti, total_cont, n_vertex):
    """
    Implementación del algoritmo de recocido simulado.
    """
    final_graph_coloreado = {nodo: 0 for nodo in graph.nodes()}
    final_valor = fun_val_coloreado(graph, final_graph_coloreado)
    final_sol = final_graph_coloreado.copy()

    Ti = Ti
    total_cont = total_cont

    new_coloreado = final_graph_coloreado.copy()
    new_val = final_valor

    # Iteración y reajuste.
    
    for cont in range(total_cont):
        T = new_T(Ti, cont, total_cont)

        nodo = random.choice(list(graph.nodes()))
        color = random.randint(1, n_vertex)
        new_coloreado[nodo] = color
        next_val = fun_val_coloreado(graph, new_coloreado)

        res_val = next_val - new_val

        if res_val < 0 or random.random() < math.exp(-res_val / T):
            new_val = next_val
            if new_val < final_valor:
                final_valor = new_val
                final_sol = new_coloreado.copy()
                final_graph_coloreado = new_coloreado.copy()
        else:
            new_coloreado = final_graph_coloreado.copy()

    return final_graph_coloreado, final_valor

def solve(archivo_txt, Ti, total_cont):
    """
    Ejecución en conjunto de todo lo desarrollado.
    """
    lines = txt_grafos_recocido(archivo_txt)
    graph, n_vertex = crear_graph(lines)
    final_graph_coloreado, final_valor = REC_SIM(graph, Ti, total_cont, n_vertex)
    colores_utilizados = len(set(final_graph_coloreado.values()))
    print("Se encontró la siguiente solución:")
    print("Número de colores:", colores_utilizados)
    for nodo, color in final_graph_coloreado.items():
        print(f"Color: {color}, vértice {nodo}")

CPU times: total: 0 ns
Wall time: 0 ns


In [90]:
%%time
solve(nodos50, Ti=1000, total_cont=1000)
solve(nodos1000, Ti=1000, total_cont=1000) # No tarda demasiado, un min y cachito.

Se encontró la siguiente solución:
Número de colores: 494
Color: 41, vértice 0
Color: 535, vértice 1
Color: 0, vértice 2
Color: 711, vértice 3
Color: 0, vértice 5
Color: 0, vértice 6
Color: 655, vértice 7
Color: 0, vértice 8
Color: 0, vértice 9
Color: 646, vértice 10
Color: 853, vértice 11
Color: 420, vértice 12
Color: 171, vértice 13
Color: 141, vértice 14
Color: 362, vértice 15
Color: 601, vértice 16
Color: 0, vértice 17
Color: 0, vértice 18
Color: 796, vértice 19
Color: 0, vértice 21
Color: 684, vértice 22
Color: 0, vértice 23
Color: 0, vértice 24
Color: 795, vértice 25
Color: 0, vértice 26
Color: 0, vértice 27
Color: 614, vértice 28
Color: 0, vértice 29
Color: 0, vértice 30
Color: 0, vértice 31
Color: 887, vértice 32
Color: 0, vértice 33
Color: 637, vértice 34
Color: 374, vértice 35
Color: 915, vértice 36
Color: 0, vértice 37
Color: 348, vértice 38
Color: 0, vértice 39
Color: 63, vértice 41
Color: 255, vértice 42
Color: 362, vértice 43
Color: 0, vértice 44
Color: 0, vértice 45
Colo

## Minimax

Como tal no se implementó Minimax, el uso del repo es inconsistente y hay propiedades privadas de los elementos del entorno

In [295]:
#!git clone https://github.com/LudwigStumpp/gym-tic-tac-toe.git

Cloning into 'gym-tic-tac-toe'...


In [1]:
%cd gym-tic-tac-toe

C:\Users\FLopezP\Desktop\PCIC\IA\gym-tic-tac-toe


C:\Users\FLopezP\Anaconda3\envs\Prueba1\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import gym

env = gym.make('gym_tictactoe:tictactoe-v0')
env.reset()
env.render()

# El tablero está etiquetado de la siguiente manera:
# |0|1|2|
# |3|4|5|
# |6|7|8|

| | | |
| | | |
| | | |


C:\Users\FLopezP\Anaconda3\envs\Prueba1\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
C:\Users\FLopezP\Anaconda3\envs\Prueba1\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
C:\Users\FLopezP\Anaconda3\envs\Prueba1\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'int'>`
  logger.warn(
C:\Users\FLopezP\Anaconda3\envs\Prueba1\lib\site-packages\gym\utils\passive_env_checker.py:272: UserWar

In [4]:
env.reset()
w, x, y, z = env.step(([0, 3]))
print(z)
env.render()

normal move player 1
| | | |
|O| | |
| | | |


In [5]:
env.reset()
for a in env.get_valid_moves():
    env.step((1,a))
    env.render()
    env.reset()
    print("---------------")

|X| | |
| | | |
| | | |
---------------
| |X| |
| | | |
| | | |
---------------
| | |X|
| | | |
| | | |
---------------
| | | |
|X| | |
| | | |
---------------
| | | |
| |X| |
| | | |
---------------
| | | |
| | |X|
| | | |
---------------
| | | |
| | | |
|X| | |
---------------
| | | |
| | | |
| |X| |
---------------
| | | |
| | | |
| | |X|
---------------


In [6]:
env1 = gym.make('gym_tictactoe:tictactoe-v0')
env1.reset()
#env1.render()

0

In [8]:
env1.reset()

for i in range(9):
    jug = i%2
    a, b, c, d = env1.step((jug, i)) # No entiendo, a veces funciona bien a veces no.
    print(d)
    
    env1.render()
    if c:
        print(f"Ganó jugador {jug+1}")
        break
    #if env1._is_full():
        #print("NO HAY ESPACIO") # Esta cosa molesta demasiado wtf

normal move player 1
|O| | |
| | | |
| | | |
normal move player 2
|O|X| |
| | | |
| | | |
normal move player 1
|O|X|O|
| | | |
| | | |
normal move player 2
|O|X|O|
|X| | |
| | | |
normal move player 1
|O|X|O|
|X|O| |
| | | |
normal move player 2
|O|X|O|
|X|O|X|
| | | |
winning move player 1
|O|X|O|
|X|O|X|
|O| | |
Ganó jugador 1


In [11]:
import random
a = random.randint(0, len(env1.get_valid_moves())-1)
print(a)

1


In [12]:
def juego_random():
    env_aux = gym.make('gym_tictactoe:tictactoe-v0')
    env_aux.reset()
    jug_inicial = 0
    while len(env_aux.get_valid_moves()) != 0:
        jug = jug_inicial % 2
        jugada = random.randint(0, len(env1.get_valid_moves())-1)
        a, b, c, d = env_aux.step((jug, jugada))
        env_aux.render()
        print("----")
        if c:
            print(f"Ganó el jugador {jug+1}")
            break

In [21]:
# Este es un ejemplo de los problemas con los que me topé a la hora de ejecutar una iteración sobre el juego.
juego_random()

AssertionError: The `info` returned by `step()` must be a python dictionary, actual type: <class 'str'>

In [15]:
env._is_full()

AttributeError: accessing private attribute '_is_full' is prohibited